[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/S-Benz/dl_cnn/blob/master/sa.ipynb)

### Install python libraries

In [12]:
!pip install wget

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import numpy as np
import pickle 
import tarfile
import wget
import os.path
import matplotlib.pyplot as plt

In [2]:
# get dict of cifar data file
# TODO: fix byte string error e.g. dict key data -> b'data' after pickle.load!!!
def batch_to_dict(file):
    with open(file, "rb") as f:
        dict = pickle.load(f, encoding="bytes")
    return dict    

In [3]:
# convert batch_data to images
def bdata_to_images(bdata):
    #  Reshape data and to float and normalize it to avoid exploding weights
    data = np.reshape(bdata, (-1,3,32,32)).astype("float32") / 255
    # Transpose data to shape (32, 32, 3) for the keras model
    data = np.transpose(data, (0, 2, 3, 1))
    return data

def map_label(label):
    living_labels = [2,3,4,5,6,7]
    return int(label in living_labels)

### Load the CIFAR-100 dataset

In [4]:
cifar_data_url = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
dl_dir = "cifar.tar.gz"
batches_dir = "cifar-batches"
train_batch_files = [batches_dir + "/data_batch_1",
               batches_dir + "/data_batch_2", 
               batches_dir + "/data_batch_3", 
               batches_dir + "/data_batch_4", 
               batches_dir + "/data_batch_5"]
test_batch_file = batches_dir + "/test_batch"

batch_meta = batches_dir + "/batches.meta"

# check if data set has been downloaded
if not os.path.exists(dl_dir):
    wget.download(cifar_data_url, dl_dir)

# extract the batches
if not os.path.exists(batches_dir):    
    tar = tarfile.open(dl_dir, "r:gz")
    for member in tar.getmembers():
        if member.isreg():
            member.name = os.path.basename(member.name)
            tar.extract(member, batches_dir)
    tar.close()

batch_dicts = {}
    
# convert train batches to dicts and save in batch_dicts
# dict_keys([b'batch_label', b'labels', b'data', b'filenames'])
for batch in train_batch_files:
    d = batch_to_dict(batch)
    k = os.path.basename(batch)
    batch_dicts[k] = d                          

### Get Train & Test Labels/Data

In [9]:
# get train data & labels
train_labels = np.array([])
train_data = np.array([])

# concatenate all train datas
train_data = np.concatenate((bdata_to_images(batch_dicts['data_batch_1'][b'data']), 
bdata_to_images(batch_dicts['data_batch_2'][b'data']),
bdata_to_images(batch_dicts['data_batch_3'][b'data']),
bdata_to_images(batch_dicts['data_batch_4'][b'data']),
bdata_to_images(batch_dicts['data_batch_5'][b'data'])), axis=0)

'''
Fragen wegen one hot encoding bei nur 2 Klassen -> 0 und 1?
'''

# concatenate all labels
for key in batch_dicts.keys(): 
    labels = np.asarray(list((map(map_label, batch_dicts[key][b'labels']))))   
    train_labels = np.append(train_labels, labels)
    
# get test data & labels
test_batch = batch_to_dict(test_batch_file)
test_data = bdata_to_images(test_batch[b'data'])
test_labels = np.asarray(list((map(map_label, test_batch[b'labels']))))

### Define CNN Model with Keras

In [48]:
model = tf.keras.models.Sequential()
# First conv layer
model.add(tf.keras.layers.Conv2D(64, (3,3), padding="same", activation="relu", input_shape=(32, 32, 3)))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))
# Second conv layer
model.add(tf.keras.layers.Conv2D(128, (3,3), padding="same", activation="relu"))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))
# Flatten the model
model.add(tf.keras.layers.Flatten())
# Dense layer
model.add(tf.keras.layers.Dense(64, activation="relu"))
model.add(tf.keras.layers.Dense(2, activation="softmax"))

In [49]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 8, 8, 128)         0         
_________________________________________________________________
dense_6 (Dense)              (None, 8, 8, 64)          8256      
_________________________________________________________________
dense_7 (Dense)              (None, 8, 8, 2)           130       
Total params: 84,034
Trainable params: 84,034
Non-trainable params: 0
__________________________________________________

### Compile & train the model

In [ ]:
# Specify the training configuration (optimizer, loss, metrics)
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001, decay=1e-6),  
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_data, train_labels, epochs=10, validation_data=(test_data, test_labels))

### Plot the results

In [ ]:
plt.plot(history.history['acc'], label='training')
plt.plot(history.history['val_acc'], label='validation')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower left')

final_loss, final_acc = model.evaluate(test_data, test_labels, verbose=2)

In [ ]:
print(final_acc)
print(final_loss)